In [0]:
import boto3
from botocore.exceptions import ClientError

In [0]:
region = 'us-east-1'
roleArn = f"arn:aws:iam::387979423286:role/dataops-databricks-assume-role-hml"
roleSessionName = f"dataops-databricks-workerenv-role-hml"

In [0]:
# https://docs.databricks.com/aws/en/connect/unity-catalog/cloud-services/use-service-credentials
boto3_session = boto3.Session(botocore_session=dbutils.credentials.getServiceCredentialsProvider('service-vinicius-teste-odin'), region_name=region)

In [0]:
STS_CLIENT = boto3_session.client('sts')

In [0]:
sts_params = {
    "RoleArn": roleArn,
    "RoleSessionName": roleSessionName,
    "DurationSeconds": 900,
}

In [0]:
sts_response = STS_CLIENT.assume_role(**sts_params).get("Credentials")

In [0]:
sts_credentials = {
    "access_key": sts_response.get("AccessKeyId"),
    "secret_key": sts_response.get("SecretAccessKey"),
    "token": sts_response.get("SessionToken"),
}

In [0]:
dynamo_client = boto3.client('dynamodb', region_name=region,
    aws_access_key_id=sts_credentials.get("access_key"),
    aws_secret_access_key=sts_credentials.get("secret_key"),
    aws_session_token=sts_credentials.get("token"))

In [0]:
def  get_dynamodb_item(table_name, data):
    response = dynamo_client.get_item(
        TableName=table_name,
        Key={
            'cpf':{'S': data.get('cpf')}  
        }
    )  
    return response

In [0]:
def  update_dynamodb_item(table_name, data, id_column = None,column=None):
    if column:
        try:
            response = dynamo_client.update_item(
                TableName=table_name,
                 Key={
                    id_column: {'S': data.get(id_column)}
                },
                UpdateExpression= f'SET {column} = :new_value',
                ExpressionAttributeValues={
                    ':new_value': {'N': str(data[column])},
                    ':status_esperado': {'N': '1'}
                },
                ConditionExpression=f'{column} = :status_esperado',
                ReturnValues='UPDATED_NEW'
            )

            return response

        except ClientError as e:
            if e.response['Error']['Code'] == 'ConditionalCheckFailedException':
                return "A condição falhou: a mudança só ocorrerá quando o usuário fizer onboarding"
            else:
                return f"Erro:\n{str(e)}"
    else:
        try:
            aux = data
            aux_updt = [f'{x} = :{x}_value' for x in aux.keys() if x != "cd_uuid"]
            aux_cond = [f'{x} <> :{x}_value' for x in aux.keys() if x != "cd_uuid"]
            updt_exp = "SET " + ','.join(aux_updt)
            cond_exp = ' OR '.join(aux_cond)
            exp_att_values = {f':{k}_value': v for k, v in aux.items() if k != "cd_uuid"}
            response = dynamo_client.update_item(
                TableName=table_name,
                Key={
                    'userId':data.get('cd_uuid')
                },
                UpdateExpression= updt_exp,
                ConditionExpression=cond_exp,
                ExpressionAttributeValues=exp_att_values,
                ReturnValues='UPDATED_NEW'
            )
            return response
        except ClientError as e:
            return f"Erro:\n{str(e)}"

In [0]:
import json
ENV = "hml" #str(dbutils.widgets.get("Env")).lower()

TABLE_NAME_ECRED = f'ecred-cpf-credit-variables-{ENV}'
column = 'onboardingCompleted'
ingestion_teste = [
  '{"cpf":"2f52731e6e1d429a18ab80afe475fbb1d03750e3e6b0fcd7fa4dc83712a473e8","onboardingCompleted":1}',
  '{"cpf":"1287cb9a4c31ef9b20f06226d3be9f2f5b80dffff67cfb1d2c26734c920ee45b","onboardingCompleted":1}',
  '{"cpf":"684f91c6b85d7f5cbac3c674ce1cd06fde630d37a01dfd3c62d973330d951f94","onboardingCompleted":1}'
]
get_dynamodb_item(TABLE_NAME_ECRED, json.loads(ingestion_teste[0]))

In [0]:
ingestion_ecred = df_json_ecred if ENV == "prd" else ingestion_teste
for i in range(len(ingestion_ecred)):
  data = json.loads(ingestion_ecred[i])
  update_dynamodb_item(TABLE_NAME_ECRED, data, id_column="cpf",column=column)